In [2]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

# Created new VoteClassifier class

In [3]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [4]:
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)

In [5]:
all_words = []

for w in movie_reviews.words():
    all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:3000]

In [6]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [7]:
# print((find_features(movie_reviews.words('neg/cv000_29416.txt'))))

In [8]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [9]:
# set that we'll train our classifier with
training_set = featuresets[:1900]

# set that we'll test against.
testing_set = featuresets[1900:]

### Original Naive Bayes

In [10]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [11]:
print("Original Naive Bayes algo accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

('Original Naive Bayes algo accuracy percent:', 68.0)


In [12]:
classifier.show_most_informative_features(15)

Most Informative Features
               insulting = True              neg : pos    =     10.5 : 1.0
            refreshingly = True              pos : neg    =      8.4 : 1.0
               uplifting = True              pos : neg    =      8.3 : 1.0
                 wasting = True              neg : pos    =      8.3 : 1.0
                    sans = True              neg : pos    =      7.6 : 1.0
              mediocrity = True              neg : pos    =      7.6 : 1.0
            effortlessly = True              pos : neg    =      7.5 : 1.0
                  fabric = True              pos : neg    =      6.4 : 1.0
             bruckheimer = True              neg : pos    =      6.3 : 1.0
                    wits = True              pos : neg    =      5.7 : 1.0
               dismissed = True              pos : neg    =      5.7 : 1.0
             overwhelmed = True              pos : neg    =      5.7 : 1.0
                   wires = True              neg : pos    =      5.6 : 1.0

### Multinomial Naive Bayes

In [13]:
MNB_classifier = SklearnClassifier(MultinomialNB())

In [14]:
MNB_classifier.train(training_set)

<SklearnClassifier(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))>

In [15]:
print("MNB_classifier algo accuracy percent:",(nltk.classify.accuracy(MNB_classifier, testing_set))*100)

('MNB_classifier algo accuracy percent:', 65.0)


### Bernoulli Naive Bayes

In [16]:
BNB_classifier = SklearnClassifier(BernoulliNB())

In [17]:
BNB_classifier.train(training_set)

<SklearnClassifier(BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))>

In [18]:
print("BNB_classifier algo accuracy percent:", (nltk.classify.accuracy(BNB_classifier, testing_set))*100)

('BNB_classifier algo accuracy percent:', 66.0)


### LogisticRegression

In [19]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)

<SklearnClassifier(LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0))>

In [20]:
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

('LogisticRegression_classifier accuracy percent:', 66.0)


### SGDClassifier_classifier

In [21]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)

<SklearnClassifier(SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))>

In [22]:
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

('SGDClassifier_classifier accuracy percent:', 59.0)


### LinearSVC_classifier

In [23]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)

<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))>

In [24]:
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

('LinearSVC_classifier accuracy percent:', 65.0)


### NuSVC_classifier

In [25]:
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)

<SklearnClassifier(NuSVC(cache_size=200, coef0=0.0, degree=3, gamma=0.0, kernel='rbf',
   max_iter=-1, nu=0.5, probability=False, random_state=None,
   shrinking=True, tol=0.001, verbose=False))>

In [26]:
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

('NuSVC_classifier accuracy percent:', 69.0)


### Using VoteClassifier

In [28]:
voted_classifier = VoteClassifier(classifier,
                                  NuSVC_classifier,
                                  LinearSVC_classifier,
                                  SGDClassifier_classifier,
                                  MNB_classifier,
                                  BNB_classifier,
                                  LogisticRegression_classifier)

In [29]:
print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)

('voted_classifier accuracy percent:', 69.0)


In [30]:
### examples
print("Classification:", voted_classifier.classify(testing_set[0][0]), "Confidence %:",voted_classifier.confidence(testing_set[0][0])*100)
print("Classification:", voted_classifier.classify(testing_set[1][0]), "Confidence %:",voted_classifier.confidence(testing_set[1][0])*100)
print("Classification:", voted_classifier.classify(testing_set[2][0]), "Confidence %:",voted_classifier.confidence(testing_set[2][0])*100)
print("Classification:", voted_classifier.classify(testing_set[3][0]), "Confidence %:",voted_classifier.confidence(testing_set[3][0])*100)
print("Classification:", voted_classifier.classify(testing_set[4][0]), "Confidence %:",voted_classifier.confidence(testing_set[4][0])*100)
print("Classification:", voted_classifier.classify(testing_set[5][0]), "Confidence %:",voted_classifier.confidence(testing_set[5][0])*100)

('Classification:', u'pos', 'Confidence %:', 0)
('Classification:', u'pos', 'Confidence %:', 0)
('Classification:', u'neg', 'Confidence %:', 100)
('Classification:', u'pos', 'Confidence %:', 100)
('Classification:', u'pos', 'Confidence %:', 0)
('Classification:', u'pos', 'Confidence %:', 100)
